In [ ]:
!pip install scikit-learn==1.0
!pip install xgboost==1.4.2
!pip install catboost==0.26.1
!pip install pandas==1.3.3
!pip install radiant-mlhub==0.3.0
!pip install rasterio==1.2.8
!pip install numpy==1.21.2
!pip install pathlib==1.0.1
!pip install tqdm==4.62.3
!pip install joblib==1.0.1
!pip install matplotlib==3.4.3
!pip install Pillow==8.3.2
!pip install torch==1.9.1
!pip install plotly==5.3.1


In [ ]:
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings('RuntimeWarning')

from radiant_mlhub import Collection
import tarfile
import os
from pathlib import Path
import json
from tqdm import tqdm

import datetime
import rasterio
import numpy as np
import pandas as pd
from collections import ChainMap
from collections import Counter
import gc
import os
from joblib import Parallel,delayed
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
import pandas as pd
import rasterio
import math
gc.collect()

9

In [ ]:
def get_date_format(month,day):
    '''
        Structures the dates in a particular format
    '''
    if (str(month)=='nan') or (str(day)=='nan'):
        return 'nan'
    else:
        if month>=10:
            if day>=10:
                return f'month_{str(int(month))}_day_{str(int(day))}'
            else:
                return f'month_{str(int(month))}_day_0{str(int(day))}'
        else:
            if day>=10:
                return f'month_0{str(int(month))}_day_{str(int(day))}'
            else:
                return f'month_0{str(int(month))}_day_0{str(int(day))}'

            
def attain_field_labels(tile_df):
    
    
    field_id_src             = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array           = field_id_src.read(1)
    
    
    return field_id_array

def get_bands(tile_df,date_time,band):
    try:
        source                   = rasterio.open(tile_df[(tile_df['dates']==date_time) & (tile_df['asset']==band)]['file_path'].values[0])
        band_array               = np.expand_dims(source.read(1).flatten(), axis=1)
        band_array               = np.reshape(band_array,(256,256))
        return band_array
        
    except:
        band_array               = np.array([np.nan]*65536)
        return band_array.reshape(256,256)


def get_aggregate_values(field_id_array):
    unique_fields          = np.unique(field_id_array)
    field_dictionary       = {}
    for fields in unique_fields:
        if fields!=0:
            
            field_assigned  = np.multiply(np.where(field_id_array==fields,1,0),field_id_array)
            
            field_assigned  = field_assigned.flatten()
            
            
            field2select    = np.unique(field_assigned)[1]
            field_distrib   = Counter(field_assigned)
            print('Field id ', fields, 'percentage ',(field_distrib[field2select])/65536)
            field_dictionary[fields] = (1000*(field_distrib[field2select]))/65536
            
    return field_dictionary       
            
        
    

In [ ]:
competition_train_df          = pd.read_csv('train_data_sentinel2.csv')
competition_train_df['month'] = pd.to_datetime(competition_train_df['datetime']).dt.month.values 
competition_train_df['day']   = pd.to_datetime(competition_train_df['datetime']).dt.day.values

competition_train_df['dates'] = competition_train_df.apply(lambda z: get_date_format(z['month'],z['day']),axis=1)

unique_dates                  = competition_train_df['dates'].unique()
unique_dates                  = np.array([z for z in unique_dates if 'nan' not in z])

print(f'Length of unique dates {len(unique_dates)}')

Length of unique dates 76


In [ ]:
date_dict                    = dict(zip(competition_train_df['datetime'].dropna().unique(),unique_dates))
date_dict                    = dict(sorted(date_dict.items(), key=lambda item: item[1]))
date_order_to_consider       = np.array(list(date_dict.values()))
tile_ids_train               = competition_train_df['tile_id'].unique()
date2parse                   = [key for key,_ in date_dict.items()]

In [ ]:
def tile_id_fetch(tile_ids,date_specific,band,regular_update_field_dict):
    if tile_ids!=1951:
        
        tile_df                    = competition_train_df[competition_train_df['tile_id']==tile_ids]
        
        field_id_array = attain_field_labels(tile_df)
        
        field_agg      = get_aggregate_values(field_id_array)
        
        return field_agg
    else:
        return {}


In [ ]:


bands = ['B01']


for band in bands:
    overall_date_dictionary = {}
    for date_specific in tqdm(date_order_to_consider[:1]):

        regular_update_field_dict      = {}

        tile_field_aggregate           = Parallel(n_jobs=-1,timeout=100000,backend="multiprocessing", verbose=1)(delayed(tile_id_fetch)(tile_ids=i,date_specific=date_specific,band=band,regular_update_field_dict=regular_update_field_dict) for i in tile_ids_train)
        _                          = regular_update_field_dict.update(dict(ChainMap(*tile_field_aggregate)))
        print(len(regular_update_field_dict))       
               

        overall_date_dictionary['size_of_field'] = regular_update_field_dict
        
        
        gc.collect()

    df = pd.DataFrame(overall_date_dictionary).reset_index().rename({'index':'Field_id'},axis=1)




  0%|          | 0/1 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.


Field id  114 percentage  0.0005645751953125
Field id  6149 percentage  0.0006256103515625
Field id  25291 percentage  0.0093231201171875
Field id  5901 percentage  0.0133209228515625
Field id  1640 percentage  6.103515625e-05
Field id  231 percentage  0.0001373291015625
Field id  14643 percentage  0.0020294189453125
Field id  4881 percentage  0.0456390380859375
Field id  4130 percentage  0.0021514892578125
Field id  4696 percentage  0.0008087158203125
Field id  246 percentage  0.0029144287109375
Field id  35013 percentage  0.0005035400390625
Field id  25186 percentage  0.0078887939453125
Field id  4542 percentage  1.52587890625e-05
Field id  11012 percentage  0.002685546875
Field id  5466 percentage  0.0136260986328125
Field id  3338 percentage  0.0175628662109375
Field id  296 percentage  0.0013427734375
Field id  5362 percentage  0.00311279296875
Field id  1934 percentage  0.032196044921875
Field id  25046 percentage  0.0030517578125
Field id  11928 percentage  0.0160064697265625
Fi

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Field id  36231 percentage  0.0020904541015625
Field id  16764 percentage  0.002044677734375
Field id  67617 percentage  0.0153045654296875
Field id  96264 percentage  0.05828857421875
Field id  890 percentage  0.0005950927734375
Field id  81142 percentage  0.000244140625
Field id  115051 percentage  0.0059356689453125
Field id  35053 percentage  0.0135345458984375
Field id  112646 percentage  0.0443267822265625
Field id  100960 percentage  0.0103607177734375
Field id  48735 percentage  0.002227783203125
Field id  91473 percentage  0.016937255859375
Field id  78315 percentage  0.002960205078125
Field id  23179 percentage  0.009796142578125
Field id  21422 percentage  0.0005950927734375
Field id  75548 percentage  0.0127105712890625
Field id  98744 percentage  0.0020294189453125
Field id  69795 percentage  0.003082275390625
Field id  41091 percentage  0.0140533447265625
Field id  111278 percentage  0.000457763671875
Field id  94108 percentage  0.0017547607421875
Field id  30291 percenta

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

Field id  109665 percentage  0.0044403076171875
Field id  101535 percentage  0.05401611328125
Field id  1546 percentage  0.0085601806640625
Field id  20518 percentage  0.002777099609375
Field id  42026 percentage  0.0064849853515625
Field id  82836 percentage  0.005584716796875
Field id  88295 percentage  4.57763671875e-05
Field id  49154 percentage  0.001953125
Field id  103691 percentage  0.026153564453125
Field id  65262 percentage  0.0089874267578125
Field id  8592 percentage  0.002777099609375
Field id  10133 percentage  0.0020904541015625
Field id  119064 percentage  0.003875732421875
Field id  12088 percentage  0.0009765625
Field id  100169 percentage  0.0021820068359375
Field id  56130 percentage  0.0442352294921875
Field id  119899 percentage  0.0249786376953125
Field id  15489 percentage  0.00048828125
Field id  117244 percentage  0.0051116943359375
Field id  24031 percentage  0.0088653564453125
Field id  67775 percentage  0.0012969970703125
Field id  116935 percentage  0.007

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Field id  33348 percentage  0.0306549072265625
Field id  98426 percentage  0.001495361328125
Field id  112887 percentage  0.002197265625
Field id  92591 percentage  0.0013427734375
Field id  104466 percentage  0.0027008056640625
Field id  110578 percentage  0.02105712890625
Field id  2843 percentage  0.004486083984375
Field id  97858 percentage  0.082733154296875
Field id  24323 percentage  0.0003814697265625
Field id  2746 percentage  0.0009307861328125
Field id  85889 percentage  0.0026702880859375
Field id  27978 percentage  3.0517578125e-05
Field id  111400 percentage  0.0055694580078125
Field id  37131 percentage  0.003021240234375
Field id  29618 percentage  0.0520477294921875
Field id  72628 percentage  0.0043182373046875
Field id  114153 percentage  0.00018310546875
Field id  2653 percentage  0.0005645751953125
Field id  94608 percentage  0.01446533203125
Field id  52035 percentage  0.0091705322265625
Field id  102513 percentage  0.0003509521484375
Field id  110634 percentage  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 1/1 [01:27<00:00, 87.66s/it]

87070


In [ ]:
df.head(10)

,Field_id,size_of_field
0,1,9.490967
1,2,0.350952
2,3,11.871338
3,4,2.304077
4,6,2.685547
5,7,6.393433
6,8,45.471191
7,9,18.539429
8,12,0.244141
9,13,1.144409


In [ ]:
df.to_csv(f'/root/sentinel2_updated/Train_data_prep/merged_train_data/size_of_field_train.csv',index=False)

In [ ]:
df['size_of_field'].describe()

count    87070.000000
mean        11.527811
std         20.558929
min          0.015259
25%          1.510620
50%          3.906250
75%         12.039185
max        509.521484
Name: size_of_field, dtype: float64